In [3]:
import tomllib  # built into Python 3.11+

with open('./uv.lock', 'rb') as lockfile:  # must open in binary mode for tomllib
    uv_data = tomllib.load(lockfile)

# Let's inspect the keys to understand the structure
for key in uv_data.keys():
    print(key)

# To find scikit-learn, loop through the 'package' list
for pkg in uv_data.get('package', []):
    if pkg.get('name') == 'scikit-learn':
        print("Found scikit-learn:")
        print(pkg)
        break
else:
    print("scikit-learn not found in uv.lock")

version
revision
requires-python
package
Found scikit-learn:
{'name': 'scikit-learn', 'version': '1.6.1', 'source': {'registry': 'https://pypi.org/simple'}, 'dependencies': [{'name': 'joblib'}, {'name': 'numpy'}, {'name': 'scipy'}, {'name': 'threadpoolctl'}], 'sdist': {'url': 'https://files.pythonhosted.org/packages/9e/a5/4ae3b3a0755f7b35a280ac90b28817d1f380318973cff14075ab41ef50d9/scikit_learn-1.6.1.tar.gz', 'hash': 'sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e', 'size': 7068312, 'upload-time': '2025-01-10T08:07:55.348Z'}, 'wheels': [{'url': 'https://files.pythonhosted.org/packages/2e/59/8eb1872ca87009bdcdb7f3cdc679ad557b992c12f4b61f9250659e592c63/scikit_learn-1.6.1-cp313-cp313-macosx_10_13_x86_64.whl', 'hash': 'sha256:2ffa1e9e25b3d93990e74a4be2c2fc61ee5af85811562f1288d5d055880c4322', 'size': 12010001, 'upload-time': '2025-01-10T08:06:58.613Z'}, {'url': 'https://files.pythonhosted.org/packages/9d/05/f2fc4effc5b32e525408524c982c468c29d22f828834f0625c5ef3d601b

In [8]:
import pickle
import warnings

warnings.filterwarnings("ignore")

# Load the pipeline model
with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

# Example record
lead = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# Get the probability of conversion
pred_proba = model.predict_proba([lead])[0, 1]
print(f"Lead conversion probability: {pred_proba:.3f}")

Lead conversion probability: 0.534


In [12]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel

# Load the pipeline model
with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

# Define input schema
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# Create FastAPI app
app = FastAPI()

# Prediction endpoint
@app.post("/predict")
def predict(client: Client):
    data = client.dict()
    proba = model.predict_proba([data])[0, 1]
    return {"subscription_probability": round(float(proba), 3)}

In [13]:
import requests

url = "http://127.0.0.1:8000/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'subscription_probability': 0.534}


In [14]:
!docker images agrigorev/zoomcamp-model

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      4a9ecc576ae9   5 days ago   121MB


In [17]:
import requests

url = "http://127.0.0.1:8000/predict" 
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())


{'subscription_probability': 0.534}
